<a href="https://colab.research.google.com/github/haeniKim/metaverse-academy/blob/master/NLP/230713_%EC%A7%88%EC%9D%98%EC%9D%91%EB%8B%B5_%EC%B6%94%EB%A1%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ratsnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.8/728.8 kB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.7 MB/s eta 0:00:00


In [ ]:
#환경설정
from ratsnlp.nlpbook.qa import QADeployArguments
args = QADeployArguments(
    pretrained_model_name = 'beomi/kobert-base',
    downstream_model_dir = './',#체크포인트 경로
    max_seq_length=128,
    max_query_length=32
)

Exception: ignored

In [ ]:
#토크나이저
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

NameError: ignored

In [ ]:
#checkpoint load
import torch
fine_tuned_model_ckpt = torch.load(
    arg.downstream_model_checkpoint_path,
    map.location = torch.device('cpu')
)

In [ ]:
#bert설정
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name
)

In [ ]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering(pretrained_model_config)

model.load_state_dict({k.replace('model.',''):v for k,v in fine_tuned_model_ckpt['state_dict'].items()})

In [ ]:
def infernce_fn(question, context):
    if question and context:
        truncated_query = tokenizer.encode(
            question,
            add_special_token=False,
            truncation=True,
            max_length=args.max_query_length
        )

        inputs = tokenizer.encode_plus(
            text=truncated_query,
            text_pair = context,
            truncation = 'only_second',
            padding = 'max_length',
            max_length = args.max_seq_length,
            return_token_type_ids=True
        )

        with torch.no_grad():
            outputs = model(**{k:torch.tensor([v]) for k,v in inputs.items()})
            start_pred = outputs.start_logits.argmax(dim=1).item()
            end_pred = outputs.end_logits.argmax(dim=1).item()
            pred_text = tokenizer.decode(inputs['input_ids'][start_pred:end_pred])

            print('정답 : ',pred_text)